In [1]:
!unzip "/content/drive/MyDrive/computervision/Gender_detection.zip"

Streaming output truncated to the last 5000 lines.
  inflating: Gender_detection/Train/male/178319.jpg.jpg  
  inflating: Gender_detection/Train/male/178321.jpg.jpg  
  inflating: Gender_detection/Train/male/178323.jpg.jpg  
  inflating: Gender_detection/Train/male/178327.jpg.jpg  
  inflating: Gender_detection/Train/male/178335.jpg.jpg  
  inflating: Gender_detection/Train/male/178349.jpg.jpg  
  inflating: Gender_detection/Train/male/178359.jpg.jpg  
  inflating: Gender_detection/Train/male/178374.jpg.jpg  
  inflating: Gender_detection/Train/male/178381.jpg.jpg  
  inflating: Gender_detection/Train/male/178383.jpg.jpg  
  inflating: Gender_detection/Train/male/178388.jpg.jpg  
  inflating: Gender_detection/Train/male/178389.jpg.jpg  
  inflating: Gender_detection/Train/male/178395.jpg.jpg  
  inflating: Gender_detection/Train/male/178397.jpg.jpg  
  inflating: Gender_detection/Train/male/178399.jpg.jpg  
  inflating: Gender_detection/Train/male/178402.jpg.jpg  
  inflating: Gender_d

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Rescaling
from tensorflow.keras.models import Sequential

# Extracting image from directory/folder
from tensorflow.keras.utils import image_dataset_from_directory


# Reading Images

In [3]:
train_path = "/content/Gender_detection/Train"
test_path = "/content/Gender_detection/Test"

train_data = image_dataset_from_directory(
    directory = train_path,
    image_size=(180,180),
    batch_size = 32
)

test_data = image_dataset_from_directory(
    directory = test_path,
    image_size = (180,180),
    batch_size = 32
)

Found 47009 files belonging to 2 classes.
Found 11649 files belonging to 2 classes.


In [4]:
train_data.class_names

['female', 'male']

# Training

In [5]:
model = Sequential()
model.add(Rescaling(1./255, input_shape=(180,180,3)))
model.add(Conv2D(16,(3,3), padding ='same'))
model.add(MaxPooling2D())
model.add(Conv2D(64,(3,3), padding='same'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(train_data, validation_data=test_data, epochs=5)

Epoch 1/5
1470/1470 [==============================] - 68s 38ms/step - loss: 0.2804 - accuracy: 0.8971 - val_loss: 0.1899 - val_accuracy: 0.9247
Epoch 2/5
1470/1470 [==============================] - 57s 39ms/step - loss: 0.1777 - accuracy: 0.9331 - val_loss: 0.1696 - val_accuracy: 0.9348
Epoch 3/5
1470/1470 [==============================] - 52s 35ms/step - loss: 0.1426 - accuracy: 0.9462 - val_loss: 0.1281 - val_accuracy: 0.9502
Epoch 4/5
1470/1470 [==============================] - 52s 36ms/step - loss: 0.1052 - accuracy: 0.9609 - val_loss: 0.1594 - val_accuracy: 0.9461
Epoch 5/5
1470/1470 [==============================] - 52s 35ms/step - loss: 0.0790 - accuracy: 0.9712 - val_loss: 0.1750 - val_accuracy: 0.9450


# Transfer Learning

In [6]:
from tensorflow.keras.applications import VGG16

In [7]:
vgg16 = VGG16(weights='imagenet', include_top=False)
vgg16.summary()

58889256/58889256 [==============================] - 2s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                              

In [ ]:
# We creating an architecture that will incorporate vgg16

model2 = Sequential()
model2.add(Rescaling(1./255, input_shape=(180,180,3)))
model2.add(vgg16)
model2.add(Flatten())
model2.add(Dense(128,activation="relu"))
model2.add(Dense(1,activation='sigmoid'))

vgg16.trainable = False
model2.compile(loss='binary_crossentropy',optimizer="adam", metrics=["accuracy"])
model2.fit(train_data, validation_data=test_data,epochs=5)

Epoch 1/5
1470/1470 [==============================] - 183s 121ms/step - loss: 0.1992 - accuracy: 0.9255 - val_loss: 0.1873 - val_accuracy: 0.9242
Epoch 2/5
1470/1470 [==============================] - 177s 120ms/step - loss: 0.1412 - accuracy: 0.9480 - val_loss: 0.1243 - val_accuracy: 0.9531
Epoch 3/5
1470/1470 [==============================] - 176s 119ms/step - loss: 0.1222 - accuracy: 0.9558 - val_loss: 0.1323 - val_accuracy: 0.9489
Epoch 4/5
1470/1470 [==============================] - 177s 120ms/step - loss: 0.1105 - accuracy: 0.9593 - val_loss: 0.1297 - val_accuracy: 0.9524
Epoch 5/5
1432/1470 [============================>.] - ETA: 3s - loss: 0.1013 - accuracy: 0.9628

# Predict Image

In [9]:
from tensorflow.keras.preprocessing import image

In [10]:
def predict_image(img_path):
  img = image.load_img(img_path,target_size=(180,180))
  img_array = image.img_to_array(img)
  img_batch = np.expand_dims(img_array, axis=0)
  result = model2.predict(img_batch)
  if result >= 0.5:
    return "male"
  else:
    return "female"

In [11]:
predict_image("/content/Gender_detection/Test/female/112961.jpg.jpg")

1/1 [==============================] - 0s 199ms/step


'female'

In [12]:
predict_image("/content/Gender_detection/Test/male/063478.jpg.jpg")

1/1 [==============================] - 0s 20ms/step


'male'

# Save model

In [13]:
model2.save("Gender_Classification.h5")

In [15]:
from google.colab import files
files.download("/content/Gender_Classification.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>